# Setting output directions

In [11]:
import numpy as np
base_dir = "./iris_results"
trial_names = np.array([
    ["GAN", "GAN"],
    ["InfoGAN", "InfoGAN"],
    ["QGAN(with MC)", "QGAN(with mode collapse)"],
    ["QGAN(no MC)", "QGAN(without mode collapse)"],
    ["InfoQGAN", "InfoQGAN"],
])


In [12]:
def calculate_max_matching_accuracy(pair_counts):
    # do a majority vote and find matching, accuracy
    
    keys = list(pair_counts.keys())
    values = list(pair_counts.values())
    species = ['setosa', 'versicolor', 'virginica']
    codes = list(set([key[1] for key in keys]))
    max_sum = 0
    best_keys = []
    for i in range(3):
        for j in range(3):
            if i==j:
                continue
            for k in range(3):
                if i==k or j==k:
                    continue
                # species[0] -> codes[i], species[1] -> codes[j], species[2] -> codes[k]
                selected_keys = [(species[0], codes[i]), (species[1], codes[j]), (species[2], codes[k])]
                cur = 0
                cur = cur + ( pair_counts[selected_keys[0]] if selected_keys[0] in pair_counts else 0 )
                cur = cur + ( pair_counts[selected_keys[1]] if selected_keys[1] in pair_counts else 0 )
                cur = cur + ( pair_counts[selected_keys[2]] if selected_keys[2] in pair_counts else 0 )
                if cur > max_sum:
                    max_sum = cur
                    best_keys = selected_keys
    return max_sum / sum(values), best_keys

# Evaluate the consistency of the model

In [ ]:
import numpy as np
import pandas as pd
import os
import kagglehub
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names")


csv_file = os.path.join("data/IRIS", "iris_train_1.csv")
raw_data_df = pd.read_csv(csv_file)  # columns: Id, SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm, Species

features = raw_data_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
target = raw_data_df["Species"]


for trial, model in trial_names:
    trial_path = os.path.join(base_dir, trial)
    scalars_path = os.path.join(trial_path, "scalars.csv")
    df = pd.read_csv(scalars_path)
    
    # 3. find the epoch that has the highest sum of p-values
    df['p_sum'] = (df['SepalLengthCm_p_value'] +
                   df['SepalWidthCm_p_value'] +
                   df['PetalLengthCm_p_value'] +
                   df['PetalWidthCm_p_value'])
    best_epoch_row = df.loc[df['p_sum'].idxmax()]
    best_epoch = int(best_epoch_row['epoch'])
    SepalLengthCm_p_value = best_epoch_row['SepalLengthCm_p_value']
    SepalWidthCm_p_value = best_epoch_row['SepalWidthCm_p_value']
    PetalLengthCm_p_value = best_epoch_row['PetalLengthCm_p_value']
    PetalWidthCm_p_value = best_epoch_row['PetalWidthCm_p_value']
    p_sum = best_epoch_row['p_sum']
    
    # 4. Read the outputs, codes file for that epoch into np.loadtxt.

    # numpy_dir = os.path.join(base_dir, trial, "numpy")
    # gen_outputs_file = os.path.join(numpy_dir, f"gen_outputs_epoch_{best_epoch}.txt")
    # gen_codes_file = os.path.join(numpy_dir, f"gen_codes_epoch_{best_epoch}.txt")
    # gen_outputs = np.loadtxt(gen_outputs_file)
    # gen_codes = np.loadtxt(gen_codes_file)

    gen_codes = np.loadtxt(os.path.join(trial_path, "codes.txt"))       # For validate purpose
    gen_outputs = np.loadtxt(os.path.join(trial_path, "outputs.txt"))   # For validate purpose
    
    acc_sum = 0
    for i in range(100): # repeat 100 times to calculate average accuracy
        # 5. Train base decision tree model
        dt_clf = DecisionTreeClassifier(random_state=42)
        dt_clf.fit(features, target)

        # 6. put gen_outputs into a decision tree model to get predicted values,
        # pair gen_codes[:, 0] with the predicted values to count the number of occurrences.
        gen_outputs_pred = dt_clf.predict(gen_outputs)
        
        pairs = list(zip(gen_outputs_pred, np.round(gen_codes[:, 0], 4)))
        pair_counts = Counter(pairs)
        acc, matching = calculate_max_matching_accuracy(pair_counts)
        acc_sum += acc
    acc = acc_sum / 100

    print(f"[{model}] Selected epoch: {best_epoch} p_sum: {p_sum} SepalLengthCm_p_value: {SepalLengthCm_p_value} SepalWidthCm_p_value: {SepalWidthCm_p_value} PetalLengthCm_p_value: {PetalLengthCm_p_value} PetalWidthCm_p_value: {PetalWidthCm_p_value} acc: {acc:.4f}")
    


[GAN] Selected epoch: 417 p_sum: 0.787201111771817 SepalLengthCm_p_value: 0.0214382736732926 SepalWidthCm_p_value: 0.7657628376405496 PetalLengthCm_p_value: 4.159654441918438e-21 PetalWidthCm_p_value: 4.579746690765983e-10 acc: 0.5781
[InfoGAN] Selected epoch: 303 p_sum: 0.8056954081082027 SepalLengthCm_p_value: 9.394321864441318e-11 SepalWidthCm_p_value: 0.8056954080142044 PetalLengthCm_p_value: 1.030023393182618e-19 PetalWidthCm_p_value: 5.5182405224738704e-14 acc: 0.8922
[QGAN(with mode collapse)] Selected epoch: 394 p_sum: 0.8740988723659426 SepalLengthCm_p_value: 0.0008021942582858 SepalWidthCm_p_value: 0.8732966780881438 PetalLengthCm_p_value: 9.122181576617383e-14 PetalWidthCm_p_value: 1.9421816275891825e-11 acc: 0.4167
[QGAN(without mode collapse)] Selected epoch: 317 p_sum: 1.9432362662849105 SepalLengthCm_p_value: 0.9070977238482086 SepalWidthCm_p_value: 0.8556104898517722 PetalLengthCm_p_value: 0.1304241313273921 PetalWidthCm_p_value: 0.0501039212575375 acc: 0.9200
[InfoQGAN

# Evaluating Augmentation Performance

In [15]:
results = {}

In [16]:
results

{}

In [ ]:
import numpy as np
import pandas as pd
import os
import kagglehub
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import Counter
import warnings

warnings.filterwarnings("ignore", message="X does not have valid feature names, but")

# Create classification model function: Creates an instance based on the desired classifier name.
def get_classifier(clf_type):
    if clf_type == "Decision Tree":
        return DecisionTreeClassifier()
    elif clf_type == "k-NN":
        return KNeighborsClassifier(n_neighbors=5)
    elif clf_type == "Logistic Regression":
        return LogisticRegression(solver="liblinear")
    else:
        raise ValueError(f"Unsupported classifier type: {clf_type}")

# Specify the desired classification model
classification_model_types = ["Decision Tree", "k-NN", "Logistic Regression"]

# Import train, test data from CSV file.
train_csv = os.path.join("data/IRIS", "iris_train_1.csv")
raw_train_df = pd.read_csv(train_csv)  
train_df = raw_train_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"]]

test_csv = os.path.join("data/IRIS", "iris_test_1.csv")
raw_test_df = pd.read_csv(test_csv)
test_df = raw_test_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"]]

# augment_settings: [number of augmented data, number of original train data].
augment_settings = np.array([
    [0, 900],
    [300, 900],
    [600, 900],
    [900, 900],
    [1200, 900],
    [1500, 900],
    [1800, 900],
])

num_iterations = 100

def evaluate_model(classification_model_type):
    for trial, model in trial_names:
        trial_path = os.path.join(base_dir, trial)
        gen_outputs = None
        gen_codes = None

        gen_outputs = np.loadtxt(os.path.join(trial_path, f"outputs_1800.txt"))
        gen_codes = np.loadtxt(os.path.join(trial_path, f"codes_1800.txt"))
        
        # round the first column of gen_codes to 4 decimal places (to avoid floating point errors)
        rounded_codes = np.round(gen_codes[:, 0], 4)
        
        # Iterate to calculate average accuracy
        for setting in augment_settings:
            base_aug_count, raw_train_count = setting  # base_aug_count: number of augmented data, raw_train_count: number of original train data
            acc_list = []
            # of augmented samples to select for each class
            aug_per_class = base_aug_count // 3 if base_aug_count > 0 else 0
            
            for iteration in range(num_iterations):
                
                origin_train = train_df.copy()
                augmented_train = origin_train.copy()
                
                if base_aug_count > 0:
                    clf1 = get_classifier(classification_model_type)
                    X_train_origin = origin_train[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
                    y_train_origin = origin_train["Species"]
                    clf1.fit(X_train_origin, y_train_origin)
                    
                    # calculate matching
                    gen_outputs_pred = clf1.predict(gen_outputs)
                    pairs = list(zip(gen_outputs_pred, rounded_codes))
                    pair_counts = Counter(pairs)
                    acc_temp, matching = calculate_max_matching_accuracy(pair_counts)
                    mapping = {code: species for species, code in matching}
                    
                    # choose samples to augment
                    augmented_samples = []
                    for code_val, species_tag in mapping.items():
                        indices = np.where(np.isclose(rounded_codes, code_val))[0]
                        if len(indices) < aug_per_class:
                            selected_indices = indices
                        else:
                            selected_indices = np.random.choice(indices, size=aug_per_class, replace=False)
                        selected_samples = gen_outputs[selected_indices]
                        df_aug = pd.DataFrame(selected_samples, 
                                            columns=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"])
                        df_aug["Species"] = species_tag
                        augmented_samples.append(df_aug)
                    if len(augmented_samples) > 0:
                        df_augmented = pd.concat(augmented_samples)
                    else:
                        df_augmented = pd.DataFrame(columns=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"])
                    # make augmented data by combining original train data and selected augmented samples
                    augmented_train = pd.concat([origin_train, df_augmented])
                
                # Classifier 2: trained by augmented data
                clf2 = get_classifier(classification_model_type)
                X_train_aug = augmented_train[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
                y_train_aug = augmented_train["Species"]
                clf2.fit(X_train_aug, y_train_aug)
                
                X_test = test_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
                y_test = test_df["Species"]
                y_pred = clf2.predict(X_test)
                acc_test = accuracy_score(y_test, y_pred)
                acc_list.append(acc_test)
            
            avg_acc = np.mean(acc_list)
            
            actual_aug_count = len(augmented_train) - raw_train_count
            if actual_aug_count % 900 == 0:
                print(f"[{model} | {trial}] [{actual_aug_count}+{raw_train_count}] | {classification_model_type} -> Avg Acc: {avg_acc:.4f}")
            results[(model, trial, classification_model_type, actual_aug_count, raw_train_count)] = avg_acc

# ---------------------------------------------------------------------------
# Final result (sorted in descending order by accuracy)
print("=====================================")

for classification_model_type in classification_model_types:
    evaluate_model(classification_model_type)

for key, acc in sorted(results.items(), key=lambda x: x[1], reverse=True):
    model_name, trial_name, clf_type, aug_count, raw_train_count = key
    print(f"[{model_name} | {trial_name} | {clf_type}] augmented: {aug_count}, raw_train: {raw_train_count} -> Avg Acc: {acc:.4f}")


[GAN | GAN] [0+900] | Decision Tree -> Avg Acc: 0.9205
[GAN | GAN] [900+900] | Decision Tree -> Avg Acc: 0.8865
[GAN | GAN] [1800+900] | Decision Tree -> Avg Acc: 0.8608
[InfoGAN | InfoGAN] [0+900] | Decision Tree -> Avg Acc: 0.9210
[InfoGAN | InfoGAN] [900+900] | Decision Tree -> Avg Acc: 0.9177
[InfoGAN | InfoGAN] [1800+900] | Decision Tree -> Avg Acc: 0.9261
[QGAN(with mode collapse) | QGAN(with MC)] [0+900] | Decision Tree -> Avg Acc: 0.9198
[QGAN(with mode collapse) | QGAN(with MC)] [900+900] | Decision Tree -> Avg Acc: 0.7471
[QGAN(with mode collapse) | QGAN(with MC)] [1800+900] | Decision Tree -> Avg Acc: 0.7062
[QGAN(without mode collapse) | QGAN(no MC)] [0+900] | Decision Tree -> Avg Acc: 0.9206
[QGAN(without mode collapse) | QGAN(no MC)] [900+900] | Decision Tree -> Avg Acc: 0.9203
[QGAN(without mode collapse) | QGAN(no MC)] [1800+900] | Decision Tree -> Avg Acc: 0.9077
[InfoQGAN | InfoQGAN] [0+900] | Decision Tree -> Avg Acc: 0.9202
[InfoQGAN | InfoQGAN] [900+900] | Decision

# display using pandas

In [20]:
import pandas as pd

data = []
for key, acc in results.items():
    model_name, trial_name, classification_model, aug_count, raw_train_count = key
    setting_str = f"{aug_count}"
    data.append({
        "model_name": model_name,
        "classification_model": classification_model,
        "setting": setting_str,
        "accuracy": acc
    })

df = pd.DataFrame(data)

df_pivot = df.pivot_table(index=["model_name", "classification_model"],
                          columns="setting", 
                          values="accuracy", 
                          aggfunc="mean")
df_pivot.reset_index(inplace=True)
df_pivot = df_pivot.reindex(columns=["model_name", "classification_model", "0", "300", "600", "900", "1200", "1500", "1800"])
df_pivot

setting,model_name,classification_model,0,300,600,900,1200,1500,1800
0,GAN,Decision Tree,0.920500,0.907033,0.898633,0.886500,0.875667,0.874000,0.860833
1,GAN,Logistic Regression,0.930000,0.865400,0.788667,0.732500,0.678267,0.637833,0.616667
2,GAN,k-NN,0.946667,0.930733,0.914800,0.900467,0.891000,0.883267,0.876667
3,InfoGAN,Decision Tree,0.921033,0.917967,0.918700,0.917733,0.917000,0.916300,0.926133
4,InfoGAN,Logistic Regression,0.930000,0.940733,0.933900,0.918433,0.904700,0.891567,0.886667
5,InfoGAN,k-NN,0.946667,0.945033,0.943667,0.943500,0.943300,0.943033,0.943333
6,InfoQGAN,Decision Tree,0.920200,0.925700,0.929067,0.931167,0.933133,0.933533,0.932233
7,InfoQGAN,Logistic Regression,0.930000,0.946567,0.956367,0.956733,0.958267,0.958133,0.953333
8,InfoQGAN,k-NN,0.946667,0.950100,0.956367,0.960967,0.963933,0.965800,0.966667
9,QGAN(with mode collapse),Decision Tree,0.919767,0.836567,0.784567,0.747067,0.715433,0.694800,0.706167


In [ ]:
df_pivot.to_csv(f"./your_path_to_csv/your_path_to_csv.csv", index=False)